## Preparation

In [1]:
%%capture
!conda install -c pykaldi pykaldi -y

In [2]:
%cd /tmp

/tmp


In [3]:
!git clone https://github.com/jimregan/fairseq/ --branch issue3581

Cloning into 'fairseq'...
remote: Enumerating objects: 28231, done.
remote: Counting objects: 100% (643/643), done.
remote: Compressing objects: 100% (365/365), done.
remote: Total 28231 (delta 342), reused 512 (delta 263), pack-reused 27588
Receiving objects: 100% (28231/28231), 11.89 MiB | 22.35 MiB/s, done.
Resolving deltas: 100% (21161/21161), done.


In [4]:
!git clone https://github.com/kpu/kenlm

Cloning into 'kenlm'...
remote: Enumerating objects: 13824, done.
remote: Counting objects: 100% (137/137), done.
remote: Compressing objects: 100% (79/79), done.
remote: Total 13824 (delta 76), reused 92 (delta 45), pack-reused 13687
Receiving objects: 100% (13824/13824), 5.49 MiB | 19.20 MiB/s, done.
Resolving deltas: 100% (7956/7956), done.


In [5]:
%%capture
!apt-get -y install libeigen3-dev liblzma-dev zlib1g-dev libbz2-dev

In [6]:
%%capture
%cd kenlm
!mkdir build
%cd build
!cmake ..
!make -j 4
%cd /tmp

In [7]:
import os
os.environ['PATH'] = f"{os.environ['PATH']}:/tmp/kenlm/build/bin/"
os.environ['FAIRSEQ_ROOT'] = '/tmp/fairseq'

In [8]:
%cd /tmp/fairseq/

/tmp/fairseq


In [9]:
%%capture
!python setup.py install

In [10]:
%cd /tmp/fairseq/examples/wav2vec/unsupervised

/tmp/fairseq/examples/wav2vec/unsupervised


In [11]:
os.environ['HYDRA_FULL_ERROR'] = '1'

## GAN

In [12]:
!ls /kaggle/input/wav2vec-u-cv-swedish-audio/

CLUS128		    precompute_pca512_cls128_mean	  train.npy
__notebook__.ipynb  precompute_pca512_cls128_mean_pooled  train.phn
__output__.json     test.lengths			  train.tsv
__results__.html    test.ltr				  train.wrd
custom.css	    test.npy				  valid.lengths
dict.test	    test.phn				  valid.ltr
dict.train	    test.tsv				  valid.npy
dict.valid	    test.wrd				  valid.phn
pca		    train.lengths			  valid.tsv
precompute_pca512   train.ltr				  valid.wrd


In [13]:
%%writefile rungan.sh
PREFIX=w2v_unsup_gan_xp
#TASK_DATA=/path/to/features/unfiltered/precompute_unfiltered_pca512_cls128_mean_pooled
TASK_DATA=/kaggle/input/wav2vec-u-cv-swedish-audio/precompute_pca512_cls128_mean_pooled/
#TEXT_DATA=/path/to/data  # path to fairseq-preprocessed GAN data
TEXT_DATA=/kaggle/input/wav2vec-u-cv-swedish-text-prep/preppedtext/phones/
#KENLM_PATH=/path/to/data/kenlm.phn.o4.bin  # KenLM 4-gram phoneme language model (LM data = GAN data here)
KENLM_PATH=/kaggle/input/wav2vec-u-cv-swedish-text-prep/preppedtext/phones/lm.phones.filtered.04.bin

PREFIX=$PREFIX fairseq-hydra-train \
	-m --config-dir config/gan \
	--config-name w2vu \
	task.data=${TASK_DATA} \
	task.text_data=${TEXT_DATA} \
	task.kenlm_path=${KENLM_PATH} \
	checkpoint.no_epoch_checkpoints=false \
	checkpoint.save_dir=/kaggle/working \
	'common.seed=range(0,5)'

Writing rungan.sh


In [14]:
!bash rungan.sh

Traceback (most recent call last):
  File "/opt/conda/bin/fairseq-hydra-train", line 33, in <module>
    sys.exit(load_entry_point('fairseq==1.0.0a0+6af4656', 'console_scripts', 'fairseq-hydra-train')())
  File "/opt/conda/lib/python3.7/site-packages/fairseq-1.0.0a0+6af4656-py3.7-linux-x86_64.egg/fairseq_cli/hydra_train.py", line 76, in cli_main
    hydra_main()
  File "/opt/conda/lib/python3.7/site-packages/hydra_core-1.0.6-py3.7.egg/hydra/main.py", line 37, in decorated_main
    strict=strict,
  File "/opt/conda/lib/python3.7/site-packages/hydra_core-1.0.6-py3.7.egg/hydra/_internal/utils.py", line 318, in _run_hydra
    run_and_report(add_conf_dir)
  File "/opt/conda/lib/python3.7/site-packages/hydra_core-1.0.6-py3.7.egg/hydra/_internal/utils.py", line 201, in run_and_report
    raise ex
  File "/opt/conda/lib/python3.7/site-packages/hydra_core-1.0.6-py3.7.egg/hydra/_internal/utils.py", line 198, in run_and_report
    return func()
  File "/opt/conda/lib/python3.7/site-packages/hydra